# Sun access 

## 0. Initialization
Importing all necessary libraries and specifying the inputs

In [1]:
import os
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np
from ladybug.sunpath import Sunpath
import pandas as pd

## 1. Import Meshes (context + envelope)

### 1.1. Load Meshes

In [2]:
envelope_path = os.path.relpath('../data/envelope.obj')
context_path = os.path.relpath('../data/immediate_context.obj')

# load the mesh from file
envelope_mesh = tm.load(envelope_path)
context_mesh = tm.load(context_path)

# Check if the mesh is watertight
print(envelope_mesh.is_watertight)
print(context_mesh.is_watertight)

faces have mixed data, using slow fallback!


True
False


### 1.2. Visualize Meshes (with pyvista)

In [3]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# initiating the plotter
p = pv.Plotter(notebook=True)

# adding the meshes
p.add_mesh(tri_to_pv(envelope_mesh), color='#abd8ff')
p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa')

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(785.6075686833789, 708.1911636833788, 743.2184808333789),
 (65.08283250000001, -12.333572500000002, 22.69374465),
 (0.0, 0.0, 1.0)]

## 2. Import Lattice (envelope)

### 2.1. Load the Envelope Lattice

In [4]:
# loading the lattice from csv
lattice_path = os.path.relpath('../data/voxelized_envelope_lowres.csv')
envelope_lattice = tg.lattice_from_csv(lattice_path)

### 2.2. Visualize the Context Mesh + Envelope Lattice

In [5]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# initiating the plotter
p = pv.Plotter(notebook=True)

# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# adding the meshes
p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa')

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(786.0557649294859, 708.6393599294859, 740.9666770318022),
 (65.08283250000001, -12.333572500000002, 19.993744602316283),
 (0.0, 0.0, 1.0)]

## 3. Sun Vectors

### 3.1. Compute Sun Vectors

In [6]:
# initiate sunpath
sp = Sunpath(longitude=4.3571, latitude=52.0116)

# define sun hours : A list of hours of the year for each sun vector
# there are 8760 hours in a year, so the following integers refer to specific hours throughout the year
hoys = []
sun_vectors = []
day_multiples = 30
for d in range(365):
    if d%day_multiples==0:
        for h in range(24):
            i = d*24 + h
            # compute the sun object
            sun = sp.calculate_sun_from_hoy(i)
            # extract the sun vector
            sun_vector = sun.sun_vector.to_array()
            # apparantly, if the Z component of sun vector is positive, 
            # it is under the horizon 
            if sun_vector[2] < 0.0:
                hoys.append(i)
                sun_vectors.append(sun_vector)
                
sun_vectors = np.array(sun_vectors)
# compute the rotation matrix 
Rz = tm.transformations.rotation_matrix(np.radians(36.324), [0,0,1])
# Rotate the sun vectors to match the site rotation
sun_vectors = tm.transform_points(sun_vectors, Rz)
print(sun_vectors.shape)

(154, 3)


In [7]:
full_lattice = envelope_lattice * 0 + 1

In [8]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# Visualize the mesh using pyvista plotter
#######

# initiating the plotter
p = pv.Plotter(notebook=True)

# fast visualization of the lattice
full_lattice.fast_vis(p)

# adding the meshes
p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa')

# add the sun locations, color orange
p.add_points( - sun_vectors * 300, color='#ffa500')

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(987.1796013214682, 980.6322311914033, 1113.9398701989232),
 (1.8922800195791183, -4.65509011048573, 128.65254889703414),
 (0.0, 0.0, 1.0)]

## 4. Compute Intersection of Sun Rays with Context Mesh

### 4.1. Preparing the List of Ray Directions and Origins

In [9]:
# constructing the sun direction from the sun vectors in a numpy array
sun_dirs = -np.array(sun_vectors)
# exract the centroids of the envelope voxels

vox_cens = full_lattice.centroids
# next step we need to shoot in all of the sun directions from all of the voxels, todo so, we need repeat the sun direction for the number of voxels to construct the ray_dir (which is the list of all ray directions). We need to repeat the voxels for the 
ray_dir = []
ray_src = []
for v_cen in vox_cens:
    for s_dir in sun_dirs:
        ray_dir.append(s_dir)
        ray_src.append(v_cen)
# converting the list of directions and sources to numpy array
ray_dir = np.array(ray_dir)
ray_src = np.array(ray_src)

print("number of voxels to shoot rays from :",vox_cens.shape)
print("number of rays per each voxel :",sun_dirs.shape)
print("number of rays to be shooted :",ray_src.shape)

number of voxels to shoot rays from : (540, 3)
number of rays per each voxel : (154, 3)
number of rays to be shooted : (83160, 3)


### 4.2. Computing the Intersection

In [10]:
# computing the intersections of rays with the context mesh
tri_id, ray_id = context_mesh.ray.intersects_id(ray_origins=ray_src, ray_directions=ray_dir, multiple_hits=False)

## 5. Aggregate Simulation Result in the Sun Access Lattice

### 5.1. Compute the percentage of time that each voxel sees the sun

In [11]:
# initializing the hits list full of zeros
hits = [0]*len(ray_dir)
# setting the rays that had an intersection to 1
for id in ray_id:
    hits[id] = 1

sun_count = len(sun_dirs)
vox_count = len(vox_cens)
# initiating the list of ratio
vox_sun_acc = []
# iterate over the voxels
for v_id in range(vox_count):
    # counter for the intersection
    int_count = 0
    # iterate over the sun rays
    for s_id in range(sun_count):
        # computing the ray id from voxel id and sun id
        r_id = v_id * sun_count + s_id

        # summing the intersections
        int_count += hits[r_id]
    
    # computing the percentage of the rays that DID NOT have 
    # an intersection (aka could see the sun)
    sun_access = 1.0 - int_count/sun_count

    # add the ratio to list
    vox_sun_acc.append(sun_access)


hits = np.array(hits)
vox_sun_acc = np.array(vox_sun_acc)

print(vox_sun_acc)

[0.57142857 0.7012987  0.85714286 0.96753247 0.61038961 0.74025974
 0.9025974  0.98051948 0.67532468 0.77922078 0.92857143 0.99350649
 0.67532468 0.80519481 0.94805195 1.         0.65584416 0.81168831
 0.96753247 1.         0.66233766 0.81168831 0.96753247 1.
 0.65584416 0.81818182 0.96753247 1.         0.6038961  0.81168831
 0.98701299 1.         0.46753247 0.7987013  0.99350649 1.
 0.51948052 0.55194805 0.82467532 0.94805195 0.48701299 0.64285714
 0.87012987 0.94805195 0.64935065 0.75974026 0.9025974  0.98051948
 0.66883117 0.77922078 0.92857143 0.98701299 0.68831169 0.7987013
 0.94805195 0.98701299 0.68831169 0.83766234 0.94805195 0.98701299
 0.69480519 0.82467532 0.94805195 0.98701299 0.66883117 0.81168831
 0.98051948 0.99350649 0.24675325 0.         0.99350649 1.
 0.54545455 0.68831169 0.81818182 0.93506494 0.52597403 0.62337662
 0.85714286 0.93506494 0.6038961  0.75974026 0.88961039 0.96753247
 0.62337662 0.75974026 0.91558442 0.98051948 0.67532468 0.78571429
 0.94155844 0.980519

### 5.2. Store sun access information in a Lattice

In [13]:
# getting the condition of all voxels: are they inside the envelop or not
env_all_vox = full_lattice.flatten()

# all voxels sun access
all_vox_sun_acc = []

# v_id: voxel id in the list of only interior voxels
v_id = 0

# for all the voxels, place the interiority condition of each voxel in "vox_in"
for vox_in in env_all_vox:
    # if the voxel was outside...
    if vox_in == True:
        # read its value of sun access and append it to the list of all voxel sun access
        all_vox_sun_acc.append(vox_sun_acc[v_id])
        # add one to the voxel id so the next time we read the next voxel
        v_id += 1
    # if the voxel was not inside... 
    else:
        # add 0.0 for its sun access
        all_vox_sun_acc.append(0.0)

# convert to array
sunacc_array = np.array(all_vox_sun_acc)

# reshape to lattice shape
sunacc_array = sunacc_array.reshape(envelope_lattice.shape)

# convert to lattice
sunacc_lattice = tg.to_lattice(sunacc_array, envelope_lattice)


print(sunacc_lattice.shape)

(15, 9, 4)


### 5.3. Visualize the sun access lattice

In [14]:
# initiating the plotter
p = pv.Plotter(notebook=True)

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape because we want to inject our values
grid.dimensions = sunacc_lattice.shape
# The bottom left corner of the data set
grid.origin = sunacc_lattice.minbound
# These are the cell sizes along each axis
grid.spacing = sunacc_lattice.unit

# Add the data values to the cell data
grid.point_arrays["Sun Access"] = sunacc_lattice.flatten(order="F")  # Flatten the Lattice

# adding the meshes
p.add_mesh(tri_to_pv(context_mesh), opacity=0.1, style='wireframe')
    
# adding the volume
opacity = np.array([0,0.6,0.6,0.6,0.6,0.6,0.6])*1.5
p.add_volume(grid, cmap="coolwarm", clim=[0.5, 1.0],opacity=opacity, shade=True)

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(785.6075686833789, 708.1911636833788, 743.2184808333789),
 (65.08283250000001, -12.333572500000002, 22.69374465),
 (0.0, 0.0, 1.0)]

## 6. Save Sun Access Lattice into a CSV

In [ ]:
# save the sun access latice to csv

csv_path = os.path.relpath('../data/sun_access_lowres.csv')
sunacc_lattice.to_csv(csv_path)

In [15]:
from scipy.interpolate import RegularGridInterpolator

In [16]:
# loading the lattice from csv
lattice_path = os.path.relpath('../data/voxelized_envelope_highres.csv')
highres_env_lattice = tg.lattice_from_csv(lattice_path)
print(highres_env_lattice.shape)

(43, 24, 11)


In [17]:
def interpolate(low_sunacc_lattice, env_lattice):
    # line spaces
    x_space = np.linspace(low_sunacc_lattice.minbound[0], low_sunacc_lattice.maxbound[0],low_sunacc_lattice.shape[0])
    y_space = np.linspace(low_sunacc_lattice.minbound[1], low_sunacc_lattice.maxbound[1],low_sunacc_lattice.shape[1])
    z_space = np.linspace(low_sunacc_lattice.minbound[2], low_sunacc_lattice.maxbound[2],low_sunacc_lattice.shape[2])

    # interpolation function
    interpolating_function = RegularGridInterpolator((x_space, y_space, z_space), low_sunacc_lattice, bounds_error=False, fill_value=None)

    # high_res lattice
    full_lattice = env_lattice + 1

    # sample points
    sample_points = full_lattice.centroids

    # interpolation
    interpolated_values = interpolating_function(sample_points)

    # lattice construction
    sunacc_lattice = tg.to_lattice(interpolated_values.reshape(env_lattice.shape), env_lattice)

    # nulling the unavailable cells
    sunacc_lattice *= env_lattice
    
    return sunacc_lattice

highres_sunacc_lattice = interpolate(sunacc_lattice,highres_env_lattice)

In [18]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

base_lattice = highres_sunacc_lattice
# initiating the plotter
p = pv.Plotter(notebook=True)

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape because we want to inject our values
grid.dimensions = base_lattice.shape
# The bottom left corner of the data set
grid.origin = base_lattice.minbound
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit

# Add the data values to the cell data
grid.point_arrays["Sun Access"] = base_lattice.flatten(order="F")  # Flatten the Lattice

# adding the meshes
# p.add_mesh(tri_to_pv(context_mesh), opacity=0.1, style='wireframe')
    
# adding the volume
opacity = np.array([0,0.6,0.6,0.6,0.6,0.6,0.6])
p.add_volume(grid, cmap="coolwarm", clim=[0.5, 1.0],opacity=opacity, shade=True)

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(232.42058333192392, 140.62058333192388, 214.4205833319239),
 (36.000000000000014, -55.800000000000004, 18.0),
 (0.0, 0.0, 1.0)]

In [21]:
# save the sun access latice to csv
csv_path = os.path.relpath('../data/sun_access_highres.csv')
highres_sunacc_lattice.to_csv(csv_path)

## 7. Envelope selection

In [22]:
# extra import function
def lattice_from_csv(file_path):
    # read metadata
    meta_df = pd.read_csv(file_path, nrows=3)

    shape = np.array(meta_df['shape'])
    unit = np.array(meta_df['unit'])
    minbound = np.array(meta_df['minbound'])

    # read lattice
    lattice_df = pd.read_csv(file_path, skiprows=5)

    # create the buffer
    buffer = np.array(lattice_df['value']).reshape(shape)

    # create the lattice
    l = tg.to_lattice(buffer, minbound=minbound, unit=unit)

    return l

In [36]:
# loading the lattice from csv
sun_acc_path = os.path.relpath('../data/sun_access_highres.csv')
sun_acc_lattice = lattice_from_csv(sun_acc_path)

### 7.1. Visualizing the selection

In [39]:
p = pv.Plotter(notebook=True)

base_lattice = sun_acc_lattice

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
#p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the avilability lattice
# init_avail_lattice.fast_vis(p)

# adding axes
#p.add_axes()
#p.show_bounds(grid="back", location="back", color="#aaaaaa")

def create_mesh(value):

    lattice = np.copy(sun_acc_lattice)
    lattice[sun_acc_lattice < value] *= 0.0
    # Add the data values to the cell data
    grid.cell_arrays["Agents"] = lattice.flatten(order="F")  # Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([0.001, 1.0])
    # adding the voxels
    p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False, clim=[0.0, 1.0])

    return

p.add_slider_widget(create_mesh, [0, 1], title='', value=0.0, event_type="always", style="classic", pointa=(0.1, 0.1), pointb=(0.9, 0.1))
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(228.98415194645204, 135.38414965763369, 209.184150444415),
 (36.00000190734863, -57.60000038146973, 16.200000405311584),
 (0.0, 0.0, 1.0)]

### 7.2. Generating an envelope based on the selection

In [40]:
#new_avail_lattice = sun_acc_lattice < threshold 

threshold = 0.551
new_avail_lattice = sun_acc_lattice > threshold

### 7.3. Visualize the new available lattice

In [41]:
p = pv.Plotter(notebook=True)

# adding the avilability lattice
new_avail_lattice.fast_vis(p)

p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(238.60252434688636, 146.8025241561515, 220.6025239892585),
 (36.0, -55.80000019073486, 17.99999964237213),
 (0.0, 0.0, 1.0)]

### 7.4. Save new envelope to CSV

In [42]:
csv_path = os.path.relpath('../data/envelope_highres_3.6.csv')
new_avail_lattice.to_csv(csv_path)

### 7.5. Low resolution selection

In [43]:
# loading the lattice from csv
sun_acc_path = os.path.relpath('../data/sun_access_lowres.csv')
sun_acc_lattice = lattice_from_csv(sun_acc_path)

In [45]:
#new_avail_lattice = sun_acc_lattice < threshold 

threshold = 0.551
new_avail_lattice = sun_acc_lattice > threshold

### 7.6. Low resolution visualise and save 

In [46]:
p = pv.Plotter(notebook=True)

# adding the avilability lattice
new_avail_lattice.fast_vis(p)

p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(248.55477740283163, 162.15477659220846, 232.35477544779928),
 (32.400001525878906, -53.99999928474426, 16.199999570846558),
 (0.0, 0.0, 1.0)]

In [47]:
csv_path = os.path.relpath('../data/envelope_lowres_10.8.csv')
new_avail_lattice.to_csv(csv_path)

### Credits

In [ ]:
__author__ = "Shervin Azadi and Pirouz Nourian"
__license__ = "MIT"
__version__ = "1.0"
__url__ = "https://github.com/shervinazadi/spatial_computing_workshops"
__summary__ = "Spatial Computing Design Studio Workshop on Solar Envelope"